# SOTA Q&A over MSMarco text (~9M chunks) in 4 minutes on Laptop

With ThirdAI we can build a Q&A system on MSMarco, the largest BEIR Benchmark — achieving SoTA accuracy in 4 minutes using just a Laptop.

In [ ]:
!pip3 install "thirdai>=0.7.40"
!pip3 install beir

### 1. Getting Started

First let's import the library and download the dataset.

In [ ]:
from thirdai import neural_db as ndb
from thirdai import demos, licensing
import pandas as pd
import time

import os
if "THIRDAI_KEY" in os.environ:
    licensing.activate(os.environ["THIRDAI_KEY"])
else:
    licensing.activate("")  # Enter your ThirdAI key here

# Downloads msmarco and puts all of the documents into a csv file.
documents, train_queries, test_queries, _ = demos.download_beir_dataset("msmarco")

### 2. Building the System

In [ ]:
qna_model = ndb.NeuralDB()

start = time.perf_counter()

qna_model.insert([
	ndb.CSV(
        documents,
        id_column="DOC_ID",        # Indicates which column stores the document ids.
        strong_columns=["TITLE"],  # Indicates which column contains the title.
        weak_columns=["TEXT"],     # Indicates which column contains the text.
)])

end = time.perf_counter()
print(f"Completed in {end-start:.3f} s")

### 3. Evaluation

In [ ]:
def parse_labels(labels):
    return list(map(int, labels.split(":")))


def evaluate(qna_model, test_queries):
    test_df = pd.read_csv(test_queries)
    test_df["DOC_ID"] = test_df["DOC_ID"].map(parse_labels)

    true_positives = 0
    start = time.perf_counter()
    for _, row in test_df.iterrows():
        result = qna_model.search(row["QUERY"], top_k=5)
        if len(result) and result[0].id in row["DOC_ID"]:
            true_positives += 1
    end = time.perf_counter()

    precision = true_positives / len(test_df)
    print(f"precision@1={precision:.3f}")
    avg_time = (end - start) / len(test_df) * 1000
    print(f"average query time: {avg_time:.3f} ms")

In [ ]:
evaluate(qna_model=qna_model, test_queries=test_queries)

So how does this compare to other systems?

| Model/System | precision@1 |
| --- | --- |
| Elastic Search  | 0.65 |
| Google T5-Base | 0.56 |
| Open AI Ada | 0.81 |
| ThirdAI | 0.79 |

### 4. Domain Specialization

While this system already achieves a comparable accuracy to the best Embedding Models, what if we want to improve it further? One of the defining characteristics of successful production search systems is their ability to continually improve based on user interactions. For example, say a company uses the custom acronym IDD to mean “initial design document”. Since this acronym doesn't appear in the training data used to create the LLMs in the search system, user queries like the “summarize the IDD for project xyz” will fail since the system doesn't understand the acronym used. With domain specialization the system can adapt to understand these relationships and answer queries like this correctly. These user interactions allow the underlying system to learn patterns/trends in user preferences that aren't present in the raw documents. 

In [ ]:
train_df = pd.read_csv(train_queries)
train_df["DOC_ID"] = train_df["DOC_ID"].map(parse_labels)

start = time.perf_counter()

qna_model.supervised_train_with_ref_ids(
    queries=train_df["QUERY"].to_list(), labels=train_df["DOC_ID"].to_list()
)

end = time.perf_counter()
print(f"finetuned in {end-start:.3f} s")

Now if we rerun the evaluation we see that the precision@1 has improved to 0.814 just by feeding sample user interactions into the system. 

In [ ]:
evaluate(qna_model=qna_model, test_queries=test_queries)

### Energy Efficient AI

Another point of emphasis should be the environmental impact. While new advances in AI are undoubtably impressive and revolutionary, the energy requirements are enormous. The next generation of Nvidia H100 GPUs in production are alone projected to surpass the energy usage of a small nation. Finding energy efficient alternatives is essential as this technology continues to develop. A system like ThirdAI that uses only a fraction of the computing resources and requires no hardware accelerators offers a path to significantly less energy usage when deploying generative AI systems.